In [ ]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

In [15]:
# get the data
import os
import pickle
import pprint as pp
import pandas as pd
import numpy as np

datadir = "./data"
# datadir = "./data_by100_backup"
fs = os.listdir(datadir)
print(fs)

['crypto_market_data_for_2000_coins_20230705_084150.pkl', 'crypto_market_data_for_3311_coins_20230710_061453.pkl', 'crypto_market_data_for_3000_coins_20230706_193546.pkl', 'crypto_market_data_for_100_coins_20230704_180439.pkl', 'crypto_market_data_for_1000_coins_20230705_004501.pkl']


In [18]:
datadir = './data'
fn = datadir + "/crypto_market_data_for_3311_coins_20230710_061453.pkl"
f = open(fn, 'rb')

data = pd.read_pickle(fn)
# Initialize a list to store DataFrames
dfs = []
# Iterate over each cryptocurrency
for crypto, crypto_data in data.items():
    if not isinstance(crypto_data, dict):
        continue

    # Convert the nested dictionary to a DataFrame
    temp_df = pd.DataFrame({
        'Date': [x[0] for x in crypto_data['market_caps']],
        'MarketCap': [x[1] for x in crypto_data['market_caps']],
    })
    
    # Convert Unix timestamp to datetime
    temp_df['Date'] = pd.to_datetime(temp_df['timestamp'], unit='ms')
    
    # Add a column for the cryptocurrency name
    temp_df['Crypto'] = crypto
    
    # Append DataFrame to the list
    dfs.append(temp_df)

# Concatenate all DataFrames
crypto_df = pd.concat(dfs, ignore_index=True)

# Filter out rows with zero market cap
crypto_df = crypto_df[crypto_df['MarketCap'] != 0]

# Filter out rows before 2017-01-01
start_date = '2017-01-01'
crypto_df = crypto_df[crypto_df['Date'] >= start_date]

df = crypto_df

,timestamp,market_cap,cryptocurrency
1342,2017-01-01,1.604599e+10,bitcoin
1343,2017-01-02,1.638793e+10,bitcoin
1344,2017-01-03,1.665272e+10,bitcoin
1345,2017-01-04,1.818766e+10,bitcoin
1346,2017-01-05,1.593489e+10,bitcoin


In [19]:
crypto_df.tail()

,timestamp,market_cap,cryptocurrency
7279202,2023-07-07 00:00:00,45304.174672,pancake-bunny
7279203,2023-07-08 00:00:00,46011.789642,pancake-bunny
7279204,2023-07-09 00:00:00,47252.051745,pancake-bunny
7279205,2023-07-10 00:00:00,41856.428192,pancake-bunny
7279206,2023-07-10 04:14:39,43972.941834,pancake-bunny


In [5]:
coins = list(d.keys())

# create mc dict
mcd = {}

for c in d.keys():
    mcd[c] = {}
    try:
        if type(d[c]) == dict:
            for m in d[c]['market_caps']:
                t = m[0]
                mv = m[1]
                mcd[c][t] = mv
    except KeyError:
        print("KeyError: ", c)



In [6]:
# create pandas dataframe
mc = pd.DataFrame(mcd)

mc["timestamp"] = pd.to_datetime(mc.index, unit='ms')
mc = mc.set_index(pd.DatetimeIndex(mc['timestamp']))
mc = mc[mc["timestamp"].dt.hour == 0]
mc = mc[mc["timestamp"].dt.minute == 0]
mc = mc[mc["timestamp"].dt.second == 0]
mc = mc.sort_index()
mc = mc.fillna(0)

In [7]:
mcf = mc[mc.index > '2017-01-01']
mcf = mcf.drop("timestamp", axis=1)

In [8]:
mcff = mcf.transpose()
#mcff.to_html("mc.html")

In [9]:
#mcff
mcff.columns
mcff.shape

(2000, 2374)

In [10]:
c = '2022-07-02'
sdf = mcff.sort_values(by=c)[c]
sdf

sci-coin-2     0.000000e+00
thala          0.000000e+00
findora        0.000000e+00
basilisk       0.000000e+00
t-mac-dao      0.000000e+00
                   ...     
binancecoin    3.575102e+10
usd-coin       5.580641e+10
tether         6.630277e+10
ethereum       1.282743e+11
bitcoin        3.709165e+11
Name: 2022-07-02 00:00:00, Length: 2000, dtype: float64

In [11]:
sd = {}
for c in mcff.columns:
    a = mcff.sort_values(by=c, ascending=False)[c]
    b = a.index
    tpl = zip(b, a)
    sd[c] = tpl

sdf = pd.DataFrame(sd)
#sdf.to_html('ranked_by_data.html')

from datetime import datetime
today = datetime.today()
date = f'{today.day}_{today.month}_{today.year}'

csv_fn = 'ranked_by_data_' + date + '.csv' 

sdf.to_csv(csv_fn)

: 

: 

In [10]:
# reverse the columns
sdf = sdf.iloc[:, ::-1]

# now show every 7th column (weekly)
sdff = sdf.iloc[:, ::7]

import dtale
d = dtale.show(sdff)
d.open_browser()

2023-05-28 12:06:03,619 - ERROR    - 10.0.0.88 - - [28/May/2023 12:06:03] code 400, message Bad request version ('úú\x13\x01\x13\x02\x13\x03À+À/À,À0Ì©Ì¨À\x13À\x14\x00\x9c\x00\x9d\x00/\x005\x01\x00\x01\x93ÊÊ\x00\x00\x00\x17\x00\x00\x00')
2023-05-28 12:06:03,637 - ERROR    - 10.0.0.88 - - [28/May/2023 12:06:03] code 400, message Bad request version ('X.£\x9e\\o¨3\x00')
2023-05-28 13:19:36,318 - INFO     - Executing shutdown due to inactivity...
2023-05-28 13:19:36,545 - INFO     - Executing shutdown...
/Users/andy/opt/anaconda3/lib/python3.9/site-packages/dtale/app.py:420: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

